In [1]:
import seaborn as sns
import metapack as mp
import rowgenerators as rg
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
from demosearch import FileCache
from pathlib import Path 

from tqdm.notebook import tqdm 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()

pkg_root = Path(mp.jupyter.open_source_package().path).parent
cache = FileCache(pkg_root.joinpath('data', 'cache'))

In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# Demosearch Census Extract
`civicknowledge.com-census-demosearch-1.3.1` Last Update: 2021-04-22T19:42:13

_Extract of select census variables for demographic radius searches_


## Building the package

Because the package can spend a long time downloading census data, a Jupyter notebook 
will time out. So, to build this package: 

1. Run `invoke build` to build CSV files in the data directory. Use `--force` to overwrite existing files.
2. Run `mp build` to build the package


 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [tracts](notebooks/Census.ipynb#tracts)**. US Census tracts
* ** [census_set](file:data/census_set.csv)**. Tract datasets combined into a single frame
* ** [agg_map](file:data/agg_map.csv)**. Description of which columns were aggregated together

## References
<ul><li> <strong>us_states</string>, <em>censusgeo://2019/5/US/state</em>. US States geographic boundaries</li><li> <strong>us_tracts_template</string>, <em>censusgeo://2019/5/{st}/tract</em>. Template URL for US tracts</li><li> <strong>B01001</string>, <em>census://2019/5/US/tract/B01001</em>. Sex by age</li><li> <strong>B01002</string>, <em>census://2019/5/US/tract/B01002</em>. Median Age by Sex</li><li> <strong>B11009</string>, <em>census://2018/5/US/tract/B11009</em>. Coupled Households by Type</li><li> <strong>B15003</string>, <em>census://2019/5/US/tract/B15003</em>. Educational Attainment</li><li> <strong>B25013</string>, <em>census://2019/5/US/tract/B25013</em>. Tenure by Educational Attainment of Householder</li><li> <strong>utm_bands</string>, <em>metapack+http://s3.amazonaws.com/radius.civicknowledge.com/civicknowledge.com-mgrs.csv#utm_bands</em>. UTM bands as polygons</li><li> <strong>utm_bands_buffered</string>, <em>metapack+http://s3.amazonaws.com/radius.civicknowledge.com/civicknowledge.com-mgrs.csv#utm_bands_buffered</em>. UTM band polygons, buffered by 10Km</li><ul>

In [3]:
from geoid.censusnames import stusab

k = 'us_tracts'

if not cache.exists(k):
    print('Writing tracts')
    
    url_t = pkg.reference('us_tracts_template').url
    frames = [rg.geoframe(url_t.format(st=st)) for st in tqdm(stusab.values())]

    tracts = pd.concat(frames).to_crs(4326)
    
    cache.put(k, tracts)

else:
    tracts = cache.get(k)
    
    

In [4]:
continential_states_fp = [ k for k,v in stusab.items() if v not in  ['HI', 'AK', 'PR', 'VI', 'MP', 'GU', 'AS']]


In [10]:
k = 'banded_us_tracks'

if True or not cache.exists(k):

    tracts['continential'] = tracts.statefp.astype(int).isin(continential_states_fp).astype(int)
    tracts['tract_id'] = tracts.sort_values('geoid').reset_index().index

    utmb = pkg.reference('utm_bands_buffered').geoframe()

    t = gpd.overlay(tracts, utmb)

    tracts = t[['geoid', 'tract_id', 'band','aland', 'awater', 'continential', 'geometry']]
    cache.put(k, tracts)

else:
    tracts = cache.get(k)
    
tracts.head()

,geoid,tract_id,band,aland,awater,continential,geometry
0,14000US01055010700,0,16,87404337,2927199,1,"POLYGON ((-85.98606 34.04525, -85.98550 34.045..."
1,14000US01055001300,1,16,5083957,836213,1,"POLYGON ((-86.00023 34.00644, -85.99888 34.009..."
2,14000US01055000900,2,16,11932347,740907,1,"POLYGON ((-86.07075 34.00036, -86.07051 34.000..."
3,14000US01055001700,3,16,9118882,0,1,"POLYGON ((-85.96884 33.98833, -85.96880 33.988..."
4,14000US01055010501,4,16,50209675,7198559,1,"POLYGON ((-86.07337 33.90699, -86.07330 33.907..."
